In [115]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime, timedelta

In [116]:
df0 = pd.read_csv("datasets/data_clean2_2019.csv")

C:\Users\david\AppData\Local\Temp\ipykernel_17616\2020476196.py:1: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  df0 = pd.read_csv("datasets/data_clean2_2019.csv")


In [117]:
df = df0.copy()

In [118]:
df

,Date,Booking ID,Requested Pickup Time,Week Day,Provider,Vehicle Type,Run,Space Type,Ambulatory,Origin Postal code,Pickup Arrival Time,Pickup Departure Time,Destination Postal code,Dropoff Arrival Time,Dropoff Departure Time,Trip Count,Origin1,Origin2,Destination1,Destination2
0,2019-01-01,10721655,09:00,2,VETS,VAN 2WK,VV401,AM,Y,L9C5W5,09:05,09:06,L8M1W9,09:19,09:19,1,L9C,5W5,L8M,1W9
1,2019-01-01,10721656,14:00,2,VETS,VAN 2WK,VV401,AM,Y,L8M1W9,NaN,NaN,L9C5W5,13:50,13:50,1,L8M,1W9,L9C,5W5
2,2019-01-01,10725059,12:00,2,VETS,VAN 2WK,VV409,WK,Y,L9C5H2,12:11,12:14,L8M1W9,12:29,12:36,1,L9C,5H2,L8M,1W9
3,2019-01-01,10725379,10:00,2,VETS,VAN 2WK,VV400,WK,Y,L9G0C1,09:29,09:31,L8M1W9,09:54,09:54,1,L9G,0C1,L8M,1W9
4,2019-01-01,10727509,13:00,2,VETS,VAN 2WK,VV401,WK,Y,L9H3R6,12:52,12:52,L9H3B7,12:52,12:55,1,L9H,3R6,L9H,3B7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
806707,2019-12-30,11941675,12:30,1,VETS,VAN 2WK,VV337,WK,Y,L8G5E4,12:29,12:29,NaN,13:17,13:18,1,L8G,5E4,NaN,NaN
806708,2019-12-30,11942341,12:30,1,DARTS,DARTS SIENNA,DS703A,AM,Y,L9B1K3,12:35,12:38,NaN,12:54,12:56,1,L9B,1K3,NaN,NaN
806709,2019-12-31,11948025,11:00,2,CMARVEL,CMARVEL VAN,CV208,WK,Y,L8L2K1,10:59,11:00,NaN,11:17,11:17,1,L8L,2K1,NaN,NaN
806710,2019-07-25,11403088,12:00,4,HTOWN,HTOWN VAN,HV124,AM,Y,NaN,12:05,12:07,NaN,12:18,12:19,1,NaN,NaN,NaN,NaN


# Calculated fields

- Requested Pickup Time: Driver can arrive 15 min before or after this time
- Pickup Arrival Time: Actual time driver arrives
- Pickup Departure Time: Time it takes for the passenger to board the 
- Dropoff Arrival Time: The time the vehicle arrives at the destination
- Dropoff Departure Time: The time it takes the client to alight

In [119]:
time_array = ['Requested Pickup Time', 'Pickup Arrival Time', 'Pickup Departure Time', 'Dropoff Arrival Time', 'Dropoff Departure Time']
df[time_array]

,Requested Pickup Time,Pickup Arrival Time,Pickup Departure Time,Dropoff Arrival Time,Dropoff Departure Time
0,09:00,09:05,09:06,09:19,09:19
1,14:00,NaN,NaN,13:50,13:50
2,12:00,12:11,12:14,12:29,12:36
3,10:00,09:29,09:31,09:54,09:54
4,13:00,12:52,12:52,12:52,12:55
...,...,...,...,...,...
806707,12:30,12:29,12:29,13:17,13:18
806708,12:30,12:35,12:38,12:54,12:56
806709,11:00,10:59,11:00,11:17,11:17
806710,12:00,12:05,12:07,12:18,12:19


In [120]:
# Convert string values to datetime format
df[time_array] = df[time_array].apply(pd.to_datetime)
#df[time_array] = df[time_array].apply(lambda x: pd.to_datetime(x, format='%H:%M'))

# Format datetime values as HH:MM time format
#df[time_array] = df[time_array].apply(lambda x: x.dt.strftime('%H:%M'))


In [121]:
df['Delay']=(df['Pickup Arrival Time'] - df['Requested Pickup Time']).dt.total_seconds() / 60
df['Board Time']=(df['Pickup Departure Time'] - df['Pickup Arrival Time']).dt.total_seconds() / 60
df['Travel Time']=(df['Dropoff Arrival Time'] -  df['Pickup Departure Time']).dt.total_seconds() / 60
df['Unboard Time']=(df['Dropoff Departure Time'] - df['Dropoff Arrival Time']).dt.total_seconds() / 60

In [135]:
calculated_array = ['Delay','Board Time','Travel Time','Unboard Time']
df[calculated_array].describe()

,Delay,Board Time,Travel Time,Unboard Time
count,755315.000000,786085.000000,779079.000000,786214.000000
mean,0.513122,3.595293,19.753684,2.148003
std,13.640628,11.404810,35.654872,12.090120
min,-100.000000,-1439.000000,-1436.000000,-1439.000000
25%,-10.000000,2.000000,10.000000,1.000000
50%,0.000000,3.000000,17.000000,2.000000
75%,10.000000,5.000000,28.000000,3.000000
max,100.000000,512.000000,346.000000,334.000000


In [136]:
for field in calculated_array:
    df.loc[(df[field]>100)|(df[field]< -100),field] = 10

In [137]:
df[calculated_array].describe()

,Delay,Board Time,Travel Time,Unboard Time
count,755315.000000,786085.000000,779079.000000,786214.000000
mean,0.513122,3.647313,20.490351,2.240407
std,13.640628,3.887556,13.551083,2.784944
min,-100.000000,-95.000000,-90.000000,-90.000000
25%,-10.000000,2.000000,10.000000,1.000000
50%,0.000000,3.000000,17.000000,2.000000
75%,10.000000,5.000000,28.000000,3.000000
max,100.000000,100.000000,100.000000,99.000000


## Delay or advance